In [1]:
import os
import re
from configparser import ConfigParser
from boto3        import session

Variable Collection

In [2]:
parser = ConfigParser()
parser.read('config.cfg')

['config.cfg']

Connection Configuration

In [3]:
conn = session.Session()
client = conn.client(
    's3',
    region_name=parser.get('aws_digital_ocean', 'REGION_NAME'),
    endpoint_url=parser.get('aws_digital_ocean', 'ENDPOINT'),
    aws_access_key_id=parser.get('aws_digital_ocean', 'ACCESS_ID'),
    aws_secret_access_key=parser.get('aws_digital_ocean', 'SECRET_KEY')
)

Pagination Activation

In [4]:
paginator = client.get_paginator('list_objects_v2')

List Prefix

In [5]:
list_pages = []
for prefix in parser['prefix_buckets']:
    bucket = parser.get('aws_digital_ocean', 'BUCKET')
    prefix_const = f"{parser.get('prefix_buckets', prefix)}"

    pages = paginator.paginate(Bucket=bucket, Prefix=prefix_const)

    for page in pages:
        if page['KeyCount'] > 0:
            for obj in page['Contents']:
                if len(re.findall(r'ALL PLATFORMS.xlsx', obj['Key']))>0:
                    list_pages.append(
                        {
                            'path_download': prefix,
                            'file_name': obj['Key']
                        }
                    )

In [6]:
print(len(list_pages))

57


In [7]:
list_pages

[{'path_download': 'contents',
  'file_name': 'Exports/2023-08-31/American Samoa/BB Media - 2023-08-31 - American Samoa - ALL PLATFORMS.xlsx'},
 {'path_download': 'contents',
  'file_name': 'Exports/2023-08-31/Australia/BB Media - 2023-08-31 - Australia - ALL PLATFORMS.xlsx'},
 {'path_download': 'contents',
  'file_name': 'Exports/2023-08-31/Bangladesh/BB Media - 2023-08-31 - Bangladesh - ALL PLATFORMS.xlsx'},
 {'path_download': 'contents',
  'file_name': 'Exports/2023-08-31/Bhutan/BB Media - 2023-08-31 - Bhutan - ALL PLATFORMS.xlsx'},
 {'path_download': 'contents',
  'file_name': 'Exports/2023-08-31/British Indian Ocean Territory/BB Media - 2023-08-31 - British Indian Ocean Territory - ALL PLATFORMS.xlsx'},
 {'path_download': 'contents',
  'file_name': 'Exports/2023-08-31/Brunei Darussalam/BB Media - 2023-08-31 - Brunei Darussalam - ALL PLATFORMS.xlsx'},
 {'path_download': 'contents',
  'file_name': 'Exports/2023-08-31/Cambodia/BB Media - 2023-08-31 - Cambodia - ALL PLATFORMS.xlsx'},


Download Files

In [8]:
for item in list_pages:
    FILE_NAME = item['file_name'].split('/')[-1]          
    PATH_DOWNLOAD = parser.get('paths', item['path_download'])

    if not os.path.exists(PATH_DOWNLOAD):
        os.makedirs(PATH_DOWNLOAD)

    FILE_DONWLOAD = f"{PATH_DOWNLOAD}/{FILE_NAME}"   
    print(FILE_NAME, PATH_DOWNLOAD, FILE_DONWLOAD)     
    with open(FILE_DONWLOAD, 'wb') as f:
        client.download_fileobj(parser.get('aws_digital_ocean', 'BUCKET'), item['file_name'], f)

BB Media - 2023-08-31 - American Samoa - ALL PLATFORMS.xlsx Datos/Contents Datos/Contents/BB Media - 2023-08-31 - American Samoa - ALL PLATFORMS.xlsx
BB Media - 2023-08-31 - Australia - ALL PLATFORMS.xlsx Datos/Contents Datos/Contents/BB Media - 2023-08-31 - Australia - ALL PLATFORMS.xlsx
BB Media - 2023-08-31 - Bangladesh - ALL PLATFORMS.xlsx Datos/Contents Datos/Contents/BB Media - 2023-08-31 - Bangladesh - ALL PLATFORMS.xlsx
BB Media - 2023-08-31 - Bhutan - ALL PLATFORMS.xlsx Datos/Contents Datos/Contents/BB Media - 2023-08-31 - Bhutan - ALL PLATFORMS.xlsx
BB Media - 2023-08-31 - British Indian Ocean Territory - ALL PLATFORMS.xlsx Datos/Contents Datos/Contents/BB Media - 2023-08-31 - British Indian Ocean Territory - ALL PLATFORMS.xlsx
BB Media - 2023-08-31 - Brunei Darussalam - ALL PLATFORMS.xlsx Datos/Contents Datos/Contents/BB Media - 2023-08-31 - Brunei Darussalam - ALL PLATFORMS.xlsx
BB Media - 2023-08-31 - Cambodia - ALL PLATFORMS.xlsx Datos/Contents Datos/Contents/BB Media - 2